In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
! pip install -U "langchain==0.2.16" langchain-community langchain-google-genai langchain langchain-chroma
! pip install -U sentence-transformers chromadb
! pip install -U pypdf

  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_google_genai-3.2.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached langchain_chroma-1.0.0-py3-none-any.whl.metadata (1.9 kB)
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_community-0.4-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_community-0.3.31-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_community-0.3.30-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_community-0.3.29-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_community-0.3.28-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_community-0.3.27-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_community-0.3.26-py3-none-any.whl.metadata (2.9 kB)
INFO: pip is still looking at multiple versions of langchain-community to determ

In [3]:
import os
from glob import glob

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [4]:
# from google.colab import userdata
# Gemini_API_key = userdata.get('Gemini_API_key')

# # 2) LLM = Gemini (qua LangChain)
# llm = ChatGoogleGenerativeAI(
#     model="gemini-2.5-flash",
#     temperature=0,
#     google_api_key=Gemini_API_key
# )

model_name = "lndat18/qwen25-1_5b-vi-alpaca-small_augmented_data"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",          # tự map GPU nếu có
    torch_dtype="auto"          # dùng kiểu phù hợp (fp16/bf16 nếu có)
)

# Tạo text-generation pipeline
gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.2,
    top_p=0.9
)

# Wrap thành LLM cho LangChain
llm = HuggingFacePipeline(pipeline=gen_pipeline)

# 3) Load lại Chroma + embedding
embedding = HuggingFaceEmbeddings(
    model_name="AITeamVN/Vietnamese_Embedding"
)

vectordb = Chroma(
    persist_directory="/content/drive/MyDrive/NLP/RAG_final/Embedding/chroma_pdfs_langchain",
    embedding_function=embedding,
)

retriever = vectordb.as_retriever(
    search_kwargs={"k": 8}
)

# 4) Prompt RAG
RAG_PROMPT = """
Bạn là trợ lý trả lời câu hỏi dựa trên các quy chế, quy định của trường đại học.

Ngữ cảnh dưới đây được trích từ tài liệu gốc (có thể nhiều đoạn, nhiều điều, chương khác nhau):

--------------------
{context}
--------------------

Yêu cầu:
- Chỉ sử dụng thông tin trong ngữ cảnh trên để trả lời.
- Trả lời ngắn gọn, rõ ràng, bằng tiếng Việt.
- Nếu ngữ cảnh không chứa đủ thông tin, hãy nói rõ là không tìm thấy quy định hoặc không đủ thông tin.

Câu hỏi của người dùng:
{input}

Câu trả lời:
"""

prompt = ChatPromptTemplate.from_template(RAG_PROMPT)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [5]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",   # hoặc "refine"/"map_reduce" tuỳ bạn
    return_source_documents=True
)


In [6]:
query = "Chương trình đào tạo của đại học Tôn Đức Thắng được tính theo đơn vị gì ?"
result = qa_chain({"query": query})

print("=== Câu trả lời ===")
print(result["result"])

print("\n=== Một số đoạn ngữ cảnh dùng ===")
for i, doc in enumerate(result["source_documents"], 1):
    print(f"\n[Đoạn {i}]")
    print(doc.page_content[:500], "...")


/tmp/ipython-input-2980053736.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = qa_chain({"query": query})


=== Câu trả lời ===
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

độ đại học tại Trường Đại học Tôn Đức Thắng (sau đây gọi tắt là Trường) theo hình thức 
tích lũy tín chỉ. 
Điều 2.  Chương trình đào tạo 
1. Chương trình đào tạo (CTĐT) được xây dựng theo đơn vị tín ch ỉ, cấu trúc từ các 
môn học hoặc học phần (gọi chung là môn học), trong đó phải có đủ các môn học bắt buộc 
và đáp ứng chuẩn chương trình đào tạo theo quy định hiện hành của Bộ Giáo dục và Đào 
tạo (GD&ĐT). Trong trường hợp đào tạo song ngành hoặc ngành chính - ngành phụ, chương 
trình đào tạo phải thể hiện rõ khối lượng học tập chung và riêng theo từng ngành.  
2. Nội dung, chuẩn đầu ra của CTĐT áp dụng chung đối với các hình thức và đối tượng 
người học khác nhau. Đối với người đã tốt nghiệp trình độ khác hoặc ngành khác, khối lượng 
học tập thực tế được xác định trên cơ sở công nhận, hoặc chuyển đổi tín 